### CNNでテキスト分類
- word2vecを利用
### @colabratory

In [1]:
!apt-get install -y -qq libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80==4.0.0b4 
!pip install chainer==4.0.0b4
!pip install gensim

Selecting previously unselected package libcusparse8.0:amd64.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../libcusparse8.0_8.0.61-1_amd64.deb ...
Unpacking libcusparse8.0:amd64 (8.0.61-1) ...
Selecting previously unselected package libnvrtc8.0:amd64.
Preparing to unpack .../libnvrtc8.0_8.0.61-1_amd64.deb ...
Unpacking libnvrtc8.0:amd64 (8.0.61-1) ...
Selecting previously unselected package libnvtoolsext1:amd64.
Preparing to unpack .../libnvtoolsext1_8.0.61-1_amd64.deb ...
Unpacking libnvtoolsext1:amd64 (8.0.61-1) ...
Setting up libnvtoolsext1:amd64 (8.0.61-1) ...
Setting up libcusparse8.0:amd64 (8.0.61-1) ...
Setting up libnvrtc8.0:amd64 (8.0.61-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
    100% |████████████████████████████████| 205.4MB 187kB/s 
    100% |████████████████████████████████| 378kB 13.1MB/s 
  Running setup.py bdist_wheel for chainer ... - \ | / - done
  Stored in directory: /root/.cache/pip/w

In [0]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.models import word2vec
import re
import codecs
from gensim import models
import gc
from sklearn.model_selection import KFold
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set(context='talk', style='darkgrid', palette='deep', font='IPAexGothic', font_scale=1, color_codes=False, rc=None)


In [3]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


### ローカルのgoogle driveのマウント

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! ls /content/gdrive/My\ Drive/Python3/document_classification/text/

20ng-test-all-terms.txt   20ng-train-no-stop.txt  reuter.txt
20ng-test-no-stop.txt	  r8-test-all-terms.txt   text_reuter1.txt
20ng-train-all-terms.txt  r8-train-all-terms.txt  text_reuter.txt


In [0]:
#データはReuters 21578（学習とテストを合体させたreuter.txt）

ID=[]
Document=[]
for line in open('./gdrive/My Drive/Python3/document_classification/text/reuter.txt', 'r'):
    itemList = line[:-1].split('\t')
    ID.append(itemList[0])
    Document.append(itemList[1].split())    

DIC = {"acq":0,"crude":1,"earn":2,"grain":3,"interest":4,"money-fx":5,"ship":6,"trade":7}
    
label = []
for i in ID:
    label.append(DIC[i])
    
ID = np.array(ID)
Document = np.array(Document)
label = np.array(label)

In [0]:
def nomarlize(vector):
    X = np.empty((len(vector), len(vector[0])))    
    for i, vec in enumerate(vector):
        j = vec / np.linalg.norm(vec)
        X[i] = j
    return X

def seikika(model):
    WORD_dic = {}
    for index, word in enumerate(model.wv.index2word):
        WORD_dic.update({word:index})#word & Index

    I_wei_norm = model.wv.syn0
    O_wei_norm = model.syn1neg
    IO_wei_norm = np.c_[model.wv.syn0, model.syn1neg]
    new_wei_norm = model.wv.syn0 + model.syn1neg
    I_wei_norm = nomarlize(I_wei_norm)
    O_wei_norm = nomarlize(O_wei_norm)
    IO_wei_norm = nomarlize(IO_wei_norm)
    new_wei_norm = nomarlize(new_wei_norm)
    
    # 単語ベクトル1,2,3,4
    return I_wei_norm,O_wei_norm,IO_wei_norm, new_wei_norm, WORD_dic

In [0]:
# 学習用文書の生成
def gen_text():
    wakati = []
    for j in X_train:
        wakati.append(' '.join(j))
    wakati = ' '.join(wakati)
    f = codecs.open('./gdrive/My Drive/Python3/document_classification/text/text_reuter1.txt', 'w',"utf-8") # 書き込みモードで開く
    f.write(wakati) # 引数の文字列をファイルに書き込む
    f.close()

In [0]:
# 文書のベクトル化
def generate_vec(wei_norm, data):
    data_x_vec = []
    for sentence in data:
        sentence_vec = []
        for word in sentence:
            try:
                if words[word] > cut_num:
                    sentence_vec.append(wei_norm[words[word]])
            except KeyError:
                pass
        data_x_vec.append(sentence_vec)
        
    # 文章の長さを揃えるため、ゼロパディングする
    max_sentence_size = 0
    for sentence_vec in data_x_vec:
        if max_sentence_size < len(sentence_vec):
            max_sentence_size = len(sentence_vec)
    for sentence_vec in data_x_vec:
        while len(sentence_vec) < max_sentence_size:
            sentence_vec.append(np.zeros(size))
    return data_x_vec, max_sentence_size

class SentenceClassifierCNN(chainer.ChainList):
    
    def __init__(self, in_channel, out_channel, filter_height_list, filter_width, out_size, max_sentence_size):
        """クラスの初期化
        
        Args:
            in_channel: 入力チャネル数
            out_channel: 出力チャネル数
            filter_height_list: フィルター縦サイズの配列
            filter_width: フィルター横サイズ
            out_size: 分類ラベル数
            max_sentence_size: 文章の長さの最大サイズ
        """
        self.filter_height_list = filter_height_list
        self.max_sentence_size = max_sentence_size
        self.convolution_num = len(filter_height_list)
        
        # Linkの定義
        link_list = [L.Convolution2D(in_channel, out_channel, (i, filter_width), pad=0) for i in filter_height_list] # Convolution層用のLinkをフィルター毎に追加
        link_list.append(L.Linear(out_channel * self.convolution_num, out_channel * self.convolution_num)) # 隠れ層
        link_list.append(L.Linear(out_channel * self.convolution_num, out_size)) # 出力層
        
        # 定義したLinkのリストを用いてクラスを初期化する
        super(SentenceClassifierCNN, self).__init__(*link_list)
        
    def __call__(self, x):
        """順伝播の計算を行う関数
        
        Args:
            x: 入力値
        Returns:
            y:
        """
        # フィルタを通した結果を格納する配列
        xcs = [None for i in self.filter_height_list]
        chs = [None for i in self.filter_height_list]
        
        # フィルタごとにループ
        for i, filter_height in enumerate(self.filter_height_list):
            xcs[i] = F.relu(self[i](x))
            chs[i] = F.max_pooling_2d(xcs[i], (self.max_sentence_size+1-filter_height))
        
        # Convolution+Poolingの結果の結合
        h = F.concat(chs, axis=2)
        h = F.dropout(F.tanh(self[self.convolution_num+0](h)))
        y = self[self.convolution_num+1](h)
        return y

In [0]:
# Word2Vecパラメータ
size = 100
window = 5
negative = 5
cut_num = 50 # 出現頻度上位単語の無視

In [10]:
# 5分割交差検証

kf = KFold(n_splits=5, shuffle=True, random_state=0)
n = 0
for train_index, test_index in kf.split(Document):
    if n <= 2:
        print(n)
        n += 1
        continue
    X_train, X_test = Document[train_index], Document[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #ID_train, ID_test = ID[train_index], ID[test_index]

    gen_text()
    gc.collect()
    
    sentences = word2vec.Text8Corpus('./gdrive/My Drive/Python3/document_classification/text/text_reuter1.txt')
    #learning
    model = word2vec.Word2Vec(sentences, size=size, min_count=5, window=window,
                              workers=8, sg = 1, seed = None,iter=15, negative=negative)
    
    I_wei_norm,O_wei_norm,IO_wei_norm,new_wei_norm, words= seikika(model)
    del model, O_wei_norm,IO_wei_norm
    gc.collect()
    WEI = [I_wei_norm, new_wei_norm]
    for index in range(len(WEI)):
        # データセット
        data_x_vec, max_sentence_size = generate_vec(WEI[index], Document)
        data_x_vec = np.array(data_x_vec, dtype="float32")
        data_t = np.array(label, dtype="int32")


        train_dataset = []
        for x, t in zip(data_x_vec[train_index], data_t[train_index]):
            train_dataset.append((x.reshape(1, max_sentence_size, size), t))
        test_dataset = []
        for x, t in zip(data_x_vec[test_index], data_t[test_index]):
            test_dataset.append((x.reshape(1, max_sentence_size, size), t))

        gpu = 0

        # 定数
        epoch_num = 12
        batch_size = 100
        out_size = 8
        filter_height_list = [1,2,3]
        out_channel = 32

        # モデルの定義
        model = L.Classifier(SentenceClassifierCNN(
            in_channel=1,
            out_channel=out_channel,
            filter_height_list=filter_height_list,
            filter_width=size,
            out_size=out_size,
            max_sentence_size=max_sentence_size
        ))

        optimizer = chainer.optimizers.Adam()
        optimizer.setup(model)

        if gpu >= 0:
            chainer.cuda.get_device(gpu).use()
            model.to_gpu(gpu)

        train = train_dataset
        test = test_dataset
        train_iter = chainer.iterators.SerialIterator(train, batch_size)
        test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False)
        updater = chainer.training.StandardUpdater(train_iter, optimizer, device=gpu)
        trainer = chainer.training.Trainer(updater, (epoch_num, "epoch"), out="result")
        trainer.extend(extensions.Evaluator(test_iter, model, device=gpu))
        trainer.extend(extensions.LogReport(trigger=(1, "epoch")))
        trainer.extend(extensions.ProgressBar()) # プログレスバー出力
        trainer.extend(extensions.PrintReport( ["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"]))
        print(index)
        trainer.run()
        del model

0
1
2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vec

0
epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           1.20881     0.829287              0.624839       0.758482                  259.504       
     total [######............................................] 13.57%
this epoch [###############################...................] 62.89%
       100 iter, 1 epoch / 12 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
2           0.624323    0.484818              0.802131       0.853304                  505.033       
3           0.366886    0.307542              0.899194       0.923393                  754.621       
     total [#############.....................................] 27.15%
this epoch [############......................................] 25.79%
       200 iter, 3 epoch / 12 epochs
   0.24837 iters/sec. Estimated time to finish: 0:36:00.822956.
4           0.243541    0.237157              0.931803       0.924732                  1000.16       
     total 